# Amazon SageMaker RL 및 RoboMaker를 사용한 분산 객체 추적기(Object Tracker) RL 훈련

---
## 개요

이 노트북에서는 [Coach](https://ai.intel.com/r-l-coach/) 툴킷의 [Clipped PPO](https://coach.nervanasys.com/algorithms/policy_optimization/cppo/index.html) 알고리즘 구현, 딥러닝 프레임 워크 인 [Tensorflow](https://www.tensorflow.org/) 및 시뮬레이션 환경으로 [AWS RoboMaker](https://console.aws.amazon.com/robomaker/home#welcome)를 사용하여 로봇(와플이라 명명) 트랙을 훈련시키고 다른 로봇(버거라 명명)을 따르는 방법을 강화 학습으로 수행하는 방법을 보여줍니다.


![Training in Action](./object-tracker-world.jpg)

---
## 어떻게 동작하나요?

강화 학습 에이전트(예: 와플)는 보상 기댓값(expected reward)을 극대화하기 위해 주어진 상태(state)에서 행동(action)을 취함으로써 주변 환경(예: 시각적 세계)과 상호 작용함으로써 버거를 추적하고 추적하는 법을 배웁니다. 에이전트는 여러 에피소드들를 통해 시행 착오(trial-and-error)를 통해 훈련에서 최적의 행동 계획을 학습합니다.

이 노트북은 SageMaker 및 **롤아웃(rollout)** 을 수행하는 두 개의 RoboMaker 시뮬레이션 환경에서 분산 RL 훈련의 예시를 보여줍니다. 현재 모델 또는 정책(policy)을 사용하여 고정된 수의 에피소드를 실행합니다. 롤아웃은 에이전트 경험(상태 전이 튜플)을 수집하고 이 데이터를 SageMaker와 공유하여 훈련합니다. SageMaker는 모델 정책을 업데이트한 후, 다음 롤아웃 시퀀스를 실행하는 데 사용됩니다. 이 훈련 루프는 모델이 수렴될 때까지 계속됩니다. 보다 엄밀하게, 우리는 다음과 같은 관점에서 문제를 정의할 수 있습니다.

1. **목표(Objective)**: 버거를 향하여 운전하는 법을 배웁니다.
2. **환경(Environment)**: AWS RoboMaker에서 호스팅되는 버거가 있는 시뮬레이터.
3. **상태(State)**: 와플 헤드 카메라로 캡처한 주행 POV 이미지.
4. **동작(Action)**: 다른 각도에서 6개의 개별 스티어링 휠 위치 (구성 가능)
5. **보상(Reward)** : 보상은 버거와의 거리에 반비례합니다. 와플이 햄버거에 가까워질수록 더 많은 보상을 받습니다. 액션이 버거에서 멀어지면 0의 보상을 얻습니다.

---
## 전제 조건(Prequisites)
### 라이브러리 임포트

시작하기 위해, 필요한 Python 라이브러리를 가져와서 권한 및 구성을 위한 몇 가지 전제 조건으로 환경을 설정합니다.

로컬 시스템 또는 SageMaker 노트북 인스턴스에서 이 노트북을 실행할 수 있습니다. 이 두 시나리오에서 다음 코드 셀들을 실행하여 `SageMaker`에서 훈련 작업을 시작하고 `RoboMaker`에서 시뮬레이션 작업을 시작할 수 있습니다.

In [1]:
import sagemaker
import boto3
import sys
import os
import glob
import re
import subprocess
from IPython.display import Markdown
import time
from time import gmtime, strftime
sys.path.append("common")
from misc import get_execution_role
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework
from markdown_helper import *

### S3 버킷 설정

In [2]:
# S3 bucket
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()
s3_output_path = 's3://{}/'.format(s3_bucket) # SDK appends the job name and output folder
print("S3 bucket path: {}".format(s3_output_path))

S3 bucket path: s3://sagemaker-us-west-2-143656149352/


### 변수 정의

훈련 작업의 작업 접두사(prefix) 및 훈련 작업과 시뮬레이션 작업 간의 동기화에 필요한 메타데이터를 저장하기 위한 s3_prefix와 같은 변수를 정의합니다.

In [3]:
# create unique job name 
job_name_prefix = 'rl-object-tracker'

# create unique job name
job_name = s3_prefix = job_name_prefix + "-sagemaker-" + strftime("%y%m%d-%H%M%S", gmtime())

# Duration of job in seconds (5 hours)
job_duration_in_seconds = 3600 * 5

aws_region = sage_session.boto_region_name
print("S3 bucket path: {}{}".format(s3_output_path, job_name))


if aws_region not in ["us-west-2", "us-east-1", "eu-west-1"]:
    raise Exception("This notebook uses RoboMaker which is available only in US East (N. Virginia), US West (Oregon) and EU (Ireland). Please switch to one of these regions.")
print("Model checkpoints and other metadata will be stored at: {}{}".format(s3_output_path, job_name))

S3 bucket path: s3://sagemaker-us-west-2-143656149352/rl-object-tracker-sagemaker-200326-072204
Model checkpoints and other metadata will be stored at: s3://sagemaker-us-west-2-143656149352/rl-object-tracker-sagemaker-200326-072204


### IAM 역할 생성

SageMaker 노트북 `role = sagemaker.get_execution_role()`을 실행할 때 실행 역할(execution role)을 얻거나 로컬 시스템에서 실행할 때 utils 메소드 `role = get_execution_role()`을 사용하여 실행 역할을 작성하세요.

In [4]:
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role('sagemaker')

print("Using IAM role arn: {}".format(role))

Using IAM role arn: arn:aws:iam::143656149352:role/service-role/AmazonSageMaker-ExecutionRole-20200323T143836


### 이 노트북에서 AWS RoboMaker를 호출하기 위한 권한 설정

이 노트북이 AWS RoboMaker 작업을 실행할 수 있게 하려면, 이 노트북의 기본 실행 역할에 하나의 신뢰 관계(trust relationshop)를 추가해야 합니다.

In [5]:
display(Markdown(generate_help_for_robomaker_trust_relationship(role)))

1. IAM 콘솔로 이동하여 현재 SageMaker 역할을 편집하세요.: [AmazonSageMaker-ExecutionRole-20200323T143836](https://console.aws.amazon.com/iam/home#/roles/AmazonSageMaker-ExecutionRole-20200323T143836).
2. 그런 다음 `Trust relationships tab`으로 이동하여 `Edit Trust Relationship`을 클릭하세요. 
3. JSON Blob을 다음으로 변경합니다.:
```json
            {
              "Version": "2012-10-17",
              "Statement": [
                {
                  "Effect": "Allow",
                  "Principal": {
                    "Service": [
                      "sagemaker.amazonaws.com",
                      "robomaker.amazonaws.com"
                    ]
                  },
                  "Action": "sts:AssumeRole"
                }
              ]
            }```
4. 완료 후, `Update Trust Policy`를 클릭하면 완료됩니다.

## VPC 구성

SageMaker와 RoboMaker는 네트워크를 통해 서로 통신해야 하므로, 이러한 서비스는 모두 VPC 모드에서 실행해야 합니다. 작업 시작 스크립트에 서브넷과 보안 그룹을 제공하면 됩니다. 이 예에서는 기본 VPC 구성을 사용합니다.

In [6]:
ec2 = boto3.client('ec2')
default_vpc = [vpc['VpcId'] for vpc in ec2.describe_vpcs()['Vpcs'] if vpc["IsDefault"] == True][0]

default_security_groups = [group["GroupId"] for group in ec2.describe_security_groups()['SecurityGroups'] \
                   if group["GroupName"] == "default" and group["VpcId"] == default_vpc]

default_subnets = [subnet["SubnetId"] for subnet in ec2.describe_subnets()["Subnets"] \
                  if subnet["VpcId"] == default_vpc and subnet['DefaultForAz']==True]

print("Using default VPC:", default_vpc)
print("Using default security group:", default_security_groups)
print("Using default subnets:", default_subnets)

Using default VPC: vpc-477fc93f
Using default security group: ['sg-f8a6d6b4']
Using default subnets: ['subnet-5ef4e604', 'subnet-5fd24774', 'subnet-db0e0da2', 'subnet-8286abc9']


VPC 모드에서 실행 중인 SageMaker 작업은 S3 리소스에 액세스할 수 없습니다. 따라서 SageMaker 컨테이너에서 S3에 액세스할 수 있도록 VPC S3 엔드포인트를 생성해야 합니다. VPC 모드에 대한 자세한 내용을 보려면 [이 링크](https://docs.aws.amazon.com/sagemaker/latest/dg/train-vpc.html)를 방문하세요.

> 아래 셀은 VPC S3 엔드포인트를 생성하기 위해 이 예제를 처음 실행하는 경우에만 실행해야 합니다.
이 예제를 처음 실행하는 경우에만 VPC S3 엔드포인트를 생성하려면 아래의 셀을 실행해야 합니다. 불완전한 권한 또는 다른 이유로 인해 실행이 실패하면, [create-s3-endpoint.md](create-s3-endpoint.md) (이 노트북과 동일한 폴더에 있음)를 따라 VPC S3 엔드포인트를 수동으로 생성하세요.

In [7]:
try:
    route_tables = [route_table["RouteTableId"] for route_table in ec2.describe_route_tables()['RouteTables']\
                if route_table['VpcId'] == default_vpc]
except Exception as e:
    if "UnauthorizedOperation" in str(e):
        display(Markdown(generate_help_for_s3_endpoint_permissions(role)))
    else:
        display(Markdown(create_s3_endpoint_manually(aws_region, default_vpc)))
    raise e

print("Trying to attach S3 endpoints to the following route tables:", route_tables)

assert len(route_tables) >= 1, "No route tables were found. Please follow the VPC S3 endpoint creation "\
                              "guide by clicking the above link."

try:
    ec2.create_vpc_endpoint(DryRun=False,
                           VpcEndpointType="Gateway",
                           VpcId=default_vpc,
                           ServiceName="com.amazonaws.{}.s3".format(aws_region),
                           RouteTableIds=route_tables)
    print("S3 endpoint created successfully!")
except Exception as e:
    if "RouteAlreadyExists" in str(e):
        print("S3 endpoint already exists.")
    elif "UnauthorizedOperation" in str(e):
        display(Markdown(generate_help_for_s3_endpoint_permissions(role)))
        raise e
    else:
        display(Markdown(create_s3_endpoint_manually(aws_region, default_vpc)))
        raise e

Trying to attach S3 endpoints to the following route tables: ['rtb-90e370eb']
S3 endpoint created successfully!


## 환경 설정

환경은 "object_tracker_env.py"라는 Python 파일에 정의되어 있으며, 파일은 `src/robomaker/environments/`에 있습니다. 이 파일은 Gazebo 기반 RoboMakersimulator의 gym 인터페이스를 구현합니다. 이것은 SageMaker와 RoboMaker에서 사용하는 공통 환경 파일입니다. 환경 변수 - `NODE_TYPE`은 코드가 실행 중인 노드를 정의합니다. 따라서 `rospy` 의존성이 있는 표현식은 RoboMaker에서만 실행됩니다.

이 파일에서 `reward_function`을 수정하여 다양한 보상 함수들을 실험할 수 있습니다. `TurtleBot3ObjectTrackerAndFollowerDiscreteEnv` 클래스의 스텝 메소드를 수정하여 행동 공간(action space) 및 조향 각도(steering angles)를 변경할 수 있습니다.

### RL 알고리즘에 대한 사전 설정 구성
RL 훈련 작업을 구성하는 파라미터는 `src/robomaker/presets/object_tracker.py`에 정의되어 있습니다. 사전 설정 파일을 사용하여 에이전트 파라메터들을 정의하여 특정 에이전트 알고리즘을 선택할 수 있습니다. 이 예에서는 Clipped PPO를 사용하는 것이 좋습니다.
이 파일을 편집하여 학습률(learning_rate), 신경망 구조, 배치 크기(batch_size), 감가율(discount factor) 등과 같은 알고리즘 매개 변수를 수정할 수 있습니다.

In [8]:
!pygmentize src/robomaker/presets/object_tracker.py

from rl_coach.agents.clipped_ppo_agent import ClippedPPOAgentParameters
from rl_coach.base_parameters import VisualizationParameters, PresetValidationParameters
from rl_coach.core_types import TrainingSteps, EnvironmentEpisodes, EnvironmentSteps, RunPhase
from rl_coach.environments.gym_environment import GymVectorEnvironment
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import ScheduleParameters
from rl_coach.schedules import LinearSchedule
from rl_coach.memories.memory import MemoryGranularity

from rl_coach.exploration_policies.categorical import CategoricalParameters
from rl_coach.filters.filter import NoInputFilter, NoOutputFilter, InputFilter
from rl_coach.filters.observation.observation_stacking_filter import ObservationStackingFilter
from rl_coach.filters.observation.observation_rgb_to_y_filter import ObservationRGBToYFilter
from rl_coach.filters.observation.observation_to_uint8_filter import Observation

### 엔트리포인트 훈련

훈련 코드는 `/src` 디렉토리에 업로드된 “training_worker.py” 파일에 작성됩니다. 고수준(high level)에서 아래 작업들을 수행합니다.

- SageMaker 노드의 IP 주소를 업로드합니다.
- 롤아웃 작업자 (RoboMaker 시뮬레이터)가 보낸 에이전트 경험을 받는 Redis 서버를 시작합니다.
- 특정 개수의 에피소드들이 수신된 후 매번 모델을 훈련시킵니다.
- S3에 신규 모델 가중치(model weight)를 업로드합니다. 그런 다음, 롤아웃 작업자는 모델을 업데이트하여 다음 에피소드 셋을 실행합니다.

In [9]:
# Uncomment the line below to see the training code
#!pygmentize src/training_worker.py

## Python SDK / 스크립트 모드를 사용하여 모델 훈련

In [10]:
s3_location = "s3://%s/%s" % (s3_bucket, s3_prefix)
!aws s3 rm --recursive {s3_location}


# Make any changes to the envrironment and preset files below and upload these files if you want to use custom environment and preset
!aws s3 cp src/robomaker/environments/ {s3_location}/environments/ --recursive --exclude ".ipynb_checkpoints*"
!aws s3 cp src/robomaker/presets/ {s3_location}/presets/ --recursive --exclude ".ipynb_checkpoints*"

upload: src/robomaker/environments/object_tracker_env.py to s3://sagemaker-us-west-2-143656149352/rl-object-tracker-sagemaker-200326-072204/environments/object_tracker_env.py
upload: src/robomaker/environments/__init__.py to s3://sagemaker-us-west-2-143656149352/rl-object-tracker-sagemaker-200326-072204/environments/__init__.py
upload: src/robomaker/presets/object_tracker.py to s3://sagemaker-us-west-2-143656149352/rl-object-tracker-sagemaker-200326-072204/presets/object_tracker.py
upload: src/robomaker/presets/__init__.py to s3://sagemaker-us-west-2-143656149352/rl-object-tracker-sagemaker-200326-072204/presets/__init__.py


먼저, Cloudwatch 로그에서 캡처하려는 다음 알고리즘 지표(metric)를 정의하여 훈련 진행 상황을 모니터링합니다. 이들은 알고리즘에 특화된 파라메터들이며 알고리즘마다 다를 수 있습니다. 이 예에서는 [Clipped PPO](https://coach.nervanasys.com/algorithms/policy_optimization/cppo/index.html)를 사용합니다.

In [11]:
metric_definitions = [
    # Training> Name=main_level/agent, Worker=0, Episode=19, Total reward=-102.88, Steps=19019, Training iteration=1
    {'Name': 'reward-training',
     'Regex': '^Training>.*Total reward=(.*?),'},
    
    # Policy training> Surrogate loss=-0.32664725184440613, KL divergence=7.255815035023261e-06, Entropy=2.83156156539917, training epoch=0, learning_rate=0.00025
    {'Name': 'ppo-surrogate-loss',
     'Regex': '^Policy training>.*Surrogate loss=(.*?),'},
     {'Name': 'ppo-entropy',
     'Regex': '^Policy training>.*Entropy=(.*?),'},
   
    # Testing> Name=main_level/agent, Worker=0, Episode=19, Total reward=1359.12, Steps=20015, Training iteration=2
    {'Name': 'reward-testing',
     'Regex': '^Testing>.*Total reward=(.*?),'},
]

RL 작업을 훈련하기 위해 RLEstimator를 사용합니다.

1. 환경, 사전 설정 및 훈련 코드가 업로드되는 소스 디렉토리를 지정합니다.
2. 엔트리포인트를 훈련 코드로 지정합니다.
3. RL 툴킷 및 프레임워크 선택을 지정합니다. RL 컨테이너의 ECR 경로로 자동 확인 가능합니다.
4. 모델 체크포인트 및 메타데이터를 저장하기 위해 인스턴스 수, 인스턴스 유형, 작업 이름, s3_bucket 및 s3_prefix와 같은 학습 파라메터들을 정의합니다. **현재 1개의 훈련 인스턴스 만 지원됩니다.**
5. 이 예에서는 RLCOACH_PRESET을 "object_tracker"로 설정합니다.
6. 로그에서 캡처하려는 지표를 정의합니다. CloudWatch 및 SageMaker Notebook에서도 시각화할 수 있습니다.

In [12]:
RLCOACH_PRESET = "object_tracker"

instance_type = "ml.c5.4xlarge"
    
estimator = RLEstimator(entry_point="training_worker.py",
                        source_dir='src',
                        dependencies=["common/sagemaker_rl"],
                        toolkit=RLToolkit.COACH,
                        toolkit_version='0.11',
                        framework=RLFramework.TENSORFLOW,
                        role=role,
                        train_instance_type=instance_type,
                        train_instance_count=1,
                        output_path=s3_output_path,
                        base_job_name=job_name_prefix,
                        train_max_run=job_duration_in_seconds,
                        hyperparameters={"s3_bucket": s3_bucket,
                                         "s3_prefix": s3_prefix,
                                         "aws_region": aws_region,
                                         "RLCOACH_PRESET": RLCOACH_PRESET,
                                      },
                        metric_definitions = metric_definitions,
                        subnets=default_subnets,
                        security_group_ids=default_security_groups,
                    )

estimator.fit(job_name=job_name, wait=False)

### Robomaker job 시작

In [13]:
from botocore.exceptions import UnknownServiceError

robomaker = boto3.client("robomaker")

### 시뮬레이션 어플리케이션 생성

먼저 `object-tracker public bundle`을 사용하여 RoboMaker 시뮬레이션 애플리케이션을 만듭니다. 이 번들에 대해 자세히 알아보거나 수정하려면 [RoboMaker 샘플 애플리케이션 Github 리포지토리](https://github.com/aws-robotics/aws-robomaker-sample-application-objecttracker)를 참조하세요.

In [14]:
bundle_s3_key = 'object-tracker/simulation_ws.tar.gz'
bundle_source = {'s3Bucket': s3_bucket,
                 's3Key': bundle_s3_key,
                 'architecture': "X86_64"}
simulation_software_suite={'name': 'Gazebo',
                           'version': '7'}
robot_software_suite={'name': 'ROS',
                      'version': 'Kinetic'}
rendering_engine={'name': 'OGRE',
                  'version': '1.x'}

In [15]:
simulation_application_bundle_location = "https://s3-us-west-2.amazonaws.com/robomaker-applications-us-west-2-11d8d0439f6a/object-tracker/object-tracker-1.0.80.0.1.0.130.0/simulation_ws.tar.gz"

!wget {simulation_application_bundle_location}
!aws s3 cp simulation_ws.tar.gz s3://{s3_bucket}/{bundle_s3_key}
!rm simulation_ws.tar.gz

--2020-03-26 07:37:15--  https://s3-us-west-2.amazonaws.com/robomaker-applications-us-west-2-11d8d0439f6a/object-tracker/object-tracker-1.0.80.0.1.0.130.0/simulation_ws.tar.gz
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.225.160
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.225.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 960265027 (916M) [application/x-tar]
Saving to: ‘simulation_ws.tar.gz’

simulation_ws.tar.g 100%[===================>] 915.78M  39.6MB/s    in 23s     

2020-03-26 07:37:38 (39.7 MB/s) - ‘simulation_ws.tar.gz’ saved [960265027/960265027]

upload: ./simulation_ws.tar.gz to s3://sagemaker-us-west-2-143656149352/object-tracker/simulation_ws.tar.gz


In [16]:
app_name = "object-tracker-sample-application" + strftime("%y%m%d-%H%M%S", gmtime())

try:
    response = robomaker.create_simulation_application(name=app_name,
                                                   sources=[bundle_source],
                                                   simulationSoftwareSuite=simulation_software_suite,
                                                   robotSoftwareSuite=robot_software_suite,
                                                   renderingEngine=rendering_engine
                                                  )
    simulation_app_arn = response["arn"]
    print("Created a new simulation app with ARN:", simulation_app_arn)
except Exception as e:
    if "AccessDeniedException" in str(e):
        display(Markdown(generate_help_for_robomaker_all_permissions(role)))
        raise e
    else:
        raise e

Created a new simulation app with ARN: arn:aws:robomaker:us-west-2:143656149352:simulation-application/object-tracker-sample-application200326-073753/1585208273118


### RoboMaker에서 시뮬레이션 작업 시작

환경을 시뮬레이션하고 훈련을 위해, 이 데이터를 SageMaker와 공유하는 [AWS RoboMaker](https://console.aws.amazon.com/robomaker/home#welcome) 시뮬레이션 작업을 생성합니다.

In [17]:
num_simulation_workers = 1

envriron_vars = {
                 "MODEL_S3_BUCKET": s3_bucket,
                 "MODEL_S3_PREFIX": s3_prefix,
                 "ROS_AWS_REGION": aws_region,
                 "MARKOV_PRESET_FILE": "object_tracker.py",
                 "NUMBER_OF_ROLLOUT_WORKERS": str(num_simulation_workers)}

simulation_application = {"application":simulation_app_arn,
                          "launchConfig": {"packageName": "object_tracker_simulation",
                                           "launchFile": "distributed_training.launch",
                                           "environmentVariables": envriron_vars}
                         }
                            
vpcConfig = {"subnets": default_subnets,
             "securityGroups": default_security_groups,
             "assignPublicIp": True}

responses = []
for job_no in range(num_simulation_workers):
    response =  robomaker.create_simulation_job(iamRole=role,
                                            clientRequestToken=strftime("%Y-%m-%d-%H-%M-%S", gmtime()),
                                            maxJobDurationInSeconds=job_duration_in_seconds,
                                            failureBehavior="Continue",
                                            simulationApplications=[simulation_application],
                                            vpcConfig=vpcConfig
                                            )
    responses.append(response)

print("Created the following jobs:")
job_arns = [response["arn"] for response in responses]
for job_arn in job_arns:
    print("Job ARN", job_arn) 

Created the following jobs:
Job ARN arn:aws:robomaker:us-west-2:143656149352:simulation-job/sim-shtj27qgbnd8


### RoboMaker에서 시뮬레이션 시각화
RoboMaker 콘솔을 방문하여 시뮬레이션을 시각화하거나 다음 셀을 실행하여 하이퍼링크를 생성할 수 있습니다.

In [19]:
display(Markdown(generate_robomaker_links(job_arns, aws_region)))

> RoboMaker Console에서 시뮬레이션 작업을 시각화하려면 다음 링크를 클릭해 주세요.
- [Simulation 1](https://us-west-2.console.aws.amazon.com/robomaker/home?region=us-west-2#simulationJobs/sim-shtj27qgbnd8)  

위 링크를 연 후 Gazebo를 클릭하면 시뮬레이터를 시작할 수 있습니다.

### 리소스 정리

RoboMaker 및 SageMaker 작업을 종료하려면 아래 셀을 실행해 주세요. 참고로, 셀을 실행하면 실행 중에 생성된 RoboMaker 리소스도 제거합니다.

In [20]:
for job_arn in job_arns:
    robomaker.cancel_simulation_job(job=job_arn)

In [21]:
sage_session.sagemaker_client.stop_training_job(TrainingJobName=estimator._current_job_name)

{'ResponseMetadata': {'RequestId': '76c3fdb5-57eb-4c61-b59d-d1cdd5a169fe',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '76c3fdb5-57eb-4c61-b59d-d1cdd5a169fe',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 26 Mar 2020 07:45:39 GMT'},
  'RetryAttempts': 0}}

### 평가

In [22]:
envriron_vars = {"MODEL_S3_BUCKET": s3_bucket,
                 "MODEL_S3_PREFIX": s3_prefix,
                 "ROS_AWS_REGION": aws_region,
                 "NUMBER_OF_TRIALS": str(20),
                 "MARKOV_PRESET_FILE": "%s.py" % RLCOACH_PRESET
                 }

simulation_application = {"application":simulation_app_arn,
                          "launchConfig": {"packageName": "object_tracker_simulation",
                                           "launchFile": "evaluation.launch",
                                           "environmentVariables": envriron_vars}
                         }
                            
vpcConfig = {"subnets": default_subnets,
             "securityGroups": default_security_groups,
             "assignPublicIp": True}



response =  robomaker.create_simulation_job(iamRole=role,
                                        clientRequestToken=strftime("%Y-%m-%d-%H-%M-%S", gmtime()),
                                        maxJobDurationInSeconds=job_duration_in_seconds,
                                        failureBehavior="Continue",
                                        simulationApplications=[simulation_application],
                                        vpcConfig=vpcConfig
                                        )
print("Created the following job:")
print("Job ARN", response["arn"])

Created the following job:
Job ARN arn:aws:robomaker:us-west-2:143656149352:simulation-job/sim-n8qs73r2vd3g


### 시뮬레이션 어플리케이션 리소스 정리

In [23]:
robomaker.delete_simulation_application(application=simulation_app_arn)

{'ResponseMetadata': {'RequestId': '85255faa-2c3d-456d-ab81-8de70def88ef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 26 Mar 2020 07:53:31 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '85255faa-2c3d-456d-ab81-8de70def88ef',
   'x-amz-apigw-id': 'J_PbUFyzvHcFaKQ=',
   'x-amzn-trace-id': 'Root=1-5e7c5f7b-b5aa3d6a152d8f0ee4f9f2e5'},
  'RetryAttempts': 0}}